In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

from pympler import asizeof #Para liberar memoria
import gc

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [ ]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

In [ ]:
#Agrupamiento por relación de trabajo
inicio = time.time()
agrupa = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
           (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
           (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
           (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
           (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
           (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
tipo = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select( agrupa, tipo, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

In [ ]:
afi['SECTOR'].unique()

In [ ]:
afi['SECTOR_A'].unique()

In [ ]:
afi.shape

In [ ]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()

In [ ]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

In [ ]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

In [ ]:
# Especifica la ruta del directorio donde quieres guardar el archivo
directorio = r_ruta

# Asegúrate de que el directorio existe
os.makedirs(directorio, exist_ok=True)

# Nombre del archivo
nombre_archivo = 'viu_clean_afi.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto1 = afi

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto1, archivo)
    
    
# Nombre del archivo
nombre_archivo = 'viu_clean_afi_fil.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto2 = afi_fil

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto2, archivo)
        
    
# #Cargar los archivos
# with open( ruta_archivo, 'rb') as archivo:
#     pickle.load(archivo)  # Carga y descarta el primer objeto
#     afi_fil = pickle.load(archivo)  # Carga el segundo objeto    

In [ ]:
#Se eliminan archivos no necesarios para liberar la memoria
del afi
del objeto1
del objeto2
del ced_counts

In [ ]:
afi_fil #Tiene una dimensión de (64638969 rows × 14 columns)

In [ ]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [ ]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Contar los meses, considerando duplicidad en los meses que puede tener aporte simultaneos
caract['MES_AS'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )
#Meses trabajados unicos
aux = afi_sel.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])
caract['MES_TU'] = caract['CEDULA_COD'].map( aux['CEDULA_COD'].value_counts() )

In [ ]:
# Nombre del archivo
nombre_archivo = 'viu_clean_caract.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto3 = caract

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto3, archivo)    

In [ ]:
del objeto3

In [ ]:
caract

In [ ]:
#Personas que tienen varios aportes simultaneos
caract[caract['MES_AS']>caract['MES_TU']]

In [ ]:
#afi_sel[afi_sel['CEDULA_COD']==21729781].tail(60)

In [ ]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [ ]:
aux = afi_sel[afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]
aux1 = afi_sel[~afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]

In [ ]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tienen más de un sector
inicio = time.time()
afi_sel_g = aux.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                            'NUM_SEC_MES': lambda x: len(x),
                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                }).reset_index()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
afi_sel_g

In [ ]:
afi_sel_g_all = pd.concat( [aux1[['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SECTOR_A', 'NUM_SEC_MES', '%_NUM_SECTOR']], 
                            afi_sel_g], ignore_index=True )

In [ ]:
afi_sel_g_all

In [ ]:
# Nombre del archivo
nombre_archivo = 'viu_clean_afi_sel_g_all.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto4 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto4, archivo)      

In [ ]:
#afi_sel_g_all = objetos[3]

In [ ]:
del aux
del aux1
del objeto4

In [ ]:
afi_sel_g_all = afi_sel_g_all.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel_g_all.loc[ (afi_sel_g_all['NUM_SEC_MES'] == 1), '%_NUM_SECTOR'] = '1'

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==216]

In [ ]:
#Se reinician los index
afi_sel_g_all.reset_index(inplace=True)
afi_sel_g_all.rename(columns={'index': 'nuevo_indice'}, inplace=True)
afi_sel_g_all.drop(columns=['nuevo_indice'], inplace=True)

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]

In [ ]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel_g_all['GRUPO'] = (afi_sel_g_all.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel_g_all['SAL_PROM_GRUPO'] = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')

In [ ]:
afi_sel_g_all

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126]['GRUPO'].unique()

In [ ]:
#Se obtienen los sueldos promedios para cada grupo y cada cedula
inicio = time.time()
df1 = afi_sel_g_all.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].first().reset_index().copy()
df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
df1

In [ ]:
 df1[df1['CEDULA_COD']==126]

In [ ]:
#Se seleccionan los 5 mejores años de sueldo
inicio = time.time()
top = 5
#top_sal = df1.groupby('CEDULA_COD')['SAL_PROM_GRUPO'].nlargest(top).reset_index(level=0, drop=True)
indices = df1.groupby('CEDULA_COD').apply(lambda x: x.index[:top]).explode()
df2 = df1.loc[indices].reset_index(drop=True)
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
df2[df2['CEDULA_COD']==126]

In [ ]:
#Seleccionar los cinco mejores años del grupo total de años de aporte
combinaciones = set(zip(df2['CEDULA_COD'], df2['GRUPO']))
afi_sel_g_all['GRUPO_SEL'] = [(cedula, grupo) in combinaciones for cedula, grupo in zip(afi_sel_g_all['CEDULA_COD'], afi_sel_g_all['GRUPO'])]
afi_sel_g_all['GRUPO_SEL'] = afi_sel_g_all['GRUPO_SEL'].astype(int)

In [ ]:
#Verificar que todas las cédulas tengan 60 registros de los salarios
ced_counts1 = afi_sel_g_all.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()
ced_counts1[ced_counts1 < 60].index.nunique()

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].tail(60)

In [ ]:
#Se crea una variable fecha para el período de mejores años y la base de cálculo
inicio = time.time()
mejores = afi_sel_g_all[afi_sel_g_all['GRUPO_SEL']==1].copy()
mejores.loc[:, 'FECHA'] = pd.to_datetime(mejores['ANIO'].astype(str) + '-' + mejores['MES'].astype(str).str.zfill(2) + '-01')

#Se calcula la base de calculo
df_prom = mejores.groupby('CEDULA_COD')['SALARIO'].mean()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
mejores

In [ ]:
afi_sel_g_all['INI_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
afi_sel_g_all['FIN_CAL'] = afi_sel_g_all['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
afi_sel_g_all['BASE_CAL'] = afi_sel_g_all['CEDULA_COD'].map( df_prom )

In [ ]:
caract['INI_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].max() )
caract['FIN_CAL'] = caract['CEDULA_COD'].map( mejores.groupby('CEDULA_COD')['FECHA'].min() )
caract['BASE_CAL'] = caract['CEDULA_COD'].map( df_prom )
caract

In [ ]:
del df1, df2, mejores

In [ ]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]['SALARIO'].mean()

In [ ]:
afi_sel_g_all[ (afi_sel_g_all['CEDULA_COD']==126)]

In [ ]:
# #     df = asignar_grupo(sect, n_grupo)
# #     df1 = valores_unicos_grupos(df)
# #     df2 = sel_top_salarios(df1, top)
# #     df = sel_grupo(df, df2)
#     df = fec_sel_grupo(df)
#     df = base_cal(df)
#     df = ati_base_cal_m1(df, inferior)
#     df = base_cal_sin_ati_m1(df)
#     df = sbu_ajuste_m1(df, SBU)

In [ ]:
#Registros con años menores al 2000
afi_sel_g_all[afi_sel_g_all['FIN_CAL'].dt.year<2000]

In [ ]:
#Se agrega el valor del SBU
SBU = pd.DataFrame( { 'ANIO':[2000,2001,2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
                              2021, 2022, 2023, 2024],
                      'VALOR':[57,85.65,105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386, 394, 400, 
                              400, 425, 450, 460]})
SBU

In [ ]:
afi_sel_g_all['SBU'] = afi_sel_g_all['ANIO'].map( SBU.groupby('ANIO')['VALOR'].first())

In [ ]:
afi_sel_g_all

In [ ]:
#Cédula que tienen su historia laboral igual al SBU
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['SALARIO'] == afi_sel_g_all['SBU']
#Si tiene -1 es un valor que no es un SBU en ese año y mes
afi_sel_g_all['ID_SBU'] = afi_sel_g_all['ID_SBU'].astype(int)-1
afi_sel_g_all

In [ ]:
#Verificar que todas las cédulas tengan 60 registros
ced_counts2 = afi_sel_g_all.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])['CEDULA_COD'].value_counts()
ced_counts2[ced_counts2 < 60].index.nunique()

In [ ]:
afi_sel_g_all[afi_sel_g_all['CEDULA_COD']==126].shape

In [ ]:
afi_sel_g_all[(afi_sel_g_all['CEDULA_COD']==126) & (afi_sel_g_all['GRUPO_SEL']==1)]

In [ ]:
# Nombre del archivo
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto5 = afi_sel_g_all

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto5, archivo)          

In [ ]:
del objeto5

In [ ]:
#Selección de cedulas con sus mejores años de sueldo y que su salario no sea un SBU
data_sin_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=-1)]
data_ati = afi_sel_g_all[ (afi_sel_g_all['GRUPO_SEL']==1) & (afi_sel_g_all['ID_SBU']!=0)]

In [ ]:
data_ati.shape #(21022610, 15)
data_sin_ati.shape #(5449506, 15)

In [ ]:
data_ati[data_ati['CEDULA_COD']==126][['CEDULA_COD','ANIO','MES','SALARIO']].shape

In [ ]:
data_ati[data_ati['CEDULA_COD']==126]

In [ ]:
# Nombre del archivo
nombre_archivo = 'viu_clean_data_ati.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto6 = data_ati

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto6, archivo)         

# Nombre del archivo
nombre_archivo = 'viu_clean_data_sin_ati.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto7 = data_sin_ati

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto7, archivo)     
    
del objeto6, objeto7

In [3]:
#Cargar los archivos
directorio = r_ruta
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo )     

In [4]:
#Análisis de atípicos
#Método de clustering jerárquico
data_l.shape

(62130167, 15)

In [5]:
data_l = data_l.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )
data_l.reset_index(inplace=True)
data_l.rename(columns={'index': 'nuevo_indice'}, inplace=True)
data_l.drop(columns=['nuevo_indice'], inplace=True)
data_l['INDICE'] = data_l.index

#Casos de no análisis
data_no_grupo = data_l[ (data_l['GRUPO_SEL']==0) ].copy()
data_no_grupo['ATI_CJ_M1'] = np.nan
data_no_grupo['ATI_CJ_M2'] = np.nan
data_no_grupo['ATI_CJ_M3'] = np.nan

#Casos de análisis
data = data_l[ (data_l['GRUPO_SEL']==1) ].copy()

#División del caso de análisis

##Primer caso--Considerando solo los salarios mayores al SBU y desde el año 2000
#data_si = data[   ( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 ) ].copy() #Para evitar las aportaciones a tiempo parcial
#data_no = data[ ~(( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() #Sueldos menores al SBU o en epoca del sucre

# ##Segundo caso--Considerando toda la historia laboral
# data_si = data #Para evitar las aportaciones a tiempo parcial
# data_no = None #Sueldos menores al SBU o en epoca del sucre

# ##Tercer caso--Considerando toda la historia laboral desde el año 2000
# data_si = data[  ( ( data['ANIO'] >= 2000 )) ] #Para evitar las aportaciones a tiempo parcial
# data_no = data[ ~( ( data['ANIO'] >= 2000 )) ] #Sueldos menores al SBU o en epoca del sucre
# data_no['ATI_CJ_M3'] = np.nan

In [ ]:
#Se crea un bucle para cada caso
for k in range(0,3):
    if (k==1):
        data_si = data[   ( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 ) ].copy() 
        data_no = data[ ~(( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() 
        data_no['ATI_CJ_M1'] = np.nan
        print('data_si', data_si.shape) #(26006294, 16)
        print('data_no',data_no.shape)  #(465822, 16)
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        
        data_val_ati = cluster_jerarquico( data_si_dic )
        data1 = extend_data( data_val_ati )
        
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M1', 'INDICE']],
                                  pd.DataFrame( data1 ), 
                                  data_no[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M1', 'INDICE']] ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar = data_jerar.reindex(data_l.index)
        
        data_l['ATI_CJ_M1'] = data_jerar['ATI_CJ_M1']
        
    if (k==2):
        data_si = data
        print('data_si', data_si.shape) 
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        data_val_ati = cluster_jerarquico( data_si_dic )
        data1 = extend_data( data_val_ati )
        
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M2', 'INDICE']],
                                  pd.DataFrame( data1 ) ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar = data_jerar.reindex(data_l.index)
        
        data_l['ATI_CJ_M2'] = data_jerar['ATI_CJ_M2']
        
    if (k==3):
        data_si = data[  ( ( data['ANIO'] >= 2000 )) ] 
        data_no = data[ ~( ( data['ANIO'] >= 2000 )) ]       
        data_no['ATI_CJ_M3'] = np.nan
        print('data_si', data_si.shape) 
        print('data_no',data_no.shape)
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        
        data_val_ati = cluster_jerarquico( data_si_dic )
        data1 = extend_data( data_val_ati )
        
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M3', 'INDICE']],
                                  pd.DataFrame( data1 ), 
                                  data_no[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M3', 'INDICE']] ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar = data_jerar.reindex(data_l.index)
        
        data_l['ATI_CJ_M3'] = data_jerar['ATI_CJ_M3']

In [6]:
print('data_l', data_l.shape)  #(62130167, 16)
print('data_no_grupo', data_no_grupo.shape)  #(35658051, 16)

#print(data_l[ (data_l['GRUPO_SEL']==0)].shape)  #(35658051, 16)
#print(data_l[ (data_l['GRUPO_SEL']==1)].shape)  #(26472116, 16)   #35658051+26472116=62130167

print('data', data.shape) #((26472116, 16)
print('data_si', data_si.shape) #(26006294, 16)
print('data_no',data_no.shape)  #(465822, 16)

data_l (62130167, 16)
data_no_grupo (35658051, 17)
data (26472116, 16)
data_si (26006294, 16)
data_no (465822, 17)


In [ ]:
26006294+465822

In [7]:
del data
gc.collect()

24

In [ ]:
data_si[data_si['CEDULA_COD']==188869]

In [8]:
inicio = time.time()
data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
#Para ejecutar el cluster jerarquico
def cluster_jerarquico( data_si_dic ):
    inicio = time.time()
    data_val_ati = {}
    for cedula in data_si_dic:

        if( len( data_si_dic[ cedula ]['SALARIO'] ) > 1 ): #Para formar al menos un cluster

            aux = np.array( data_si_dic[ cedula ]['SALARIO'] ).reshape(-1, 1)
            Z = linkage( aux , method='single', metric='euclidean')

            num_clusters = 2  # Puedes ajustar este valor según tus necesidades
            clusters = fcluster(Z, num_clusters, criterion='maxclust')

            if( len( np.unique(clusters) ) > 1 ): #Para considerar al menos 2 cluster
                # Calcular el centroide de cada clúster
                cluster_centers = np.array([[ np.nanmean(aux[clusters == i], axis=0)[0], i ] for i in range(1, num_clusters + 1)] ) 

                Q1 = np.quantile(aux, 0.25)
                Q3 = np.quantile(aux, 0.75)
                IQR = Q3-Q1
                LI = Q1 - 1.5 * IQR
                LS = Q3 + 1.5 * IQR

                cl_at = np.where( cluster_centers[:,0] > (LS +  1e-8) )[0]
                cluster_centers[cl_at][:, 1]

                mod_aux = np.zeros((len(aux), 2))
                mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
                mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1
                mod_aux

                data_val_ati[cedula] = {'SALARIO': mod_aux[:, 0].tolist(),
                                        'ATI_CJ_M1': mod_aux[:, 1].tolist(),
                                        'INDICE' : data_si_dic[ cedula ]['INDICE']}
            else:
                data_val_ati[cedula] = { 'SALARIO': aux.flatten().tolist(),
                                         'ATI_CJ_M1':  [-1] * len(aux),
                                         'INDICE' : data_si_dic[ cedula ]['INDICE']} 

        else:
            data_val_ati[cedula] = { 'SALARIO': data_si_dic[ cedula ]['SALARIO'],
                                     'ATI_CJ_M1':  [-1] * len( data_si_dic[ cedula ]['SALARIO'] ),
                                     'INDICE' : data_si_dic[ cedula ]['INDICE']}
    fin = time.time()  
    print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    
    return data_val_ati
 
#Para unir los resultados
def extend_data( data_val_ati ):
    inicio = time.time()
    data1 = { 'CEDULA_COD': [], 'SALARIO': [], 'ATI_CJ_M1': [], 'INDICE':[]}

    # Llenar las listas con los datos del diccionario
    for cedula, values in data_val_ati.items():
        salario = values['SALARIO']
        atipico = values['ATI_CJ_M1']
        indice = values['INDICE']
        num_rows = len(salario)

        # Extender las listas en el diccionario de datos
        data1['CEDULA_COD'].extend([cedula] * num_rows)
        data1['SALARIO'].extend(salario)
        data1['ATI_CJ_M1'].extend(atipico)
        data1['INDICE'].extend(indice)

    fin = time.time()  
    print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    return data1

In [9]:
#Algoritmo jerarquico
inicio = time.time()
data_val_ati = {}

for cedula in data_si_dic:
    
    if( len( data_si_dic[ cedula ]['SALARIO'] ) > 1 ): #Para formar al menos un cluster
        
        aux = np.array( data_si_dic[ cedula ]['SALARIO'] ).reshape(-1, 1)
        Z = linkage( aux , method='single', metric='euclidean')

        num_clusters = 2  # Puedes ajustar este valor según tus necesidades
        clusters = fcluster(Z, num_clusters, criterion='maxclust')

        if( len( np.unique(clusters) ) > 1 ): #Para considerar al menos 2 cluster
            # Calcular el centroide de cada clúster
            cluster_centers = np.array([[ np.nanmean(aux[clusters == i], axis=0)[0], i ] for i in range(1, num_clusters + 1)] ) 
                    
            Q1 = np.quantile(aux, 0.25)
            Q3 = np.quantile(aux, 0.75)
            IQR = Q3-Q1
            LI = Q1 - 1.5 * IQR
            LS = Q3 + 1.5 * IQR

            cl_at = np.where( cluster_centers[:,0] > (LS +  1e-8) )[0]
            cluster_centers[cl_at][:, 1]

            mod_aux = np.zeros((len(aux), 2))
            mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
            mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1
            mod_aux

            data_val_ati[cedula] = {'SALARIO': mod_aux[:, 0].tolist(),
                                    'ATI_CJ_M1': mod_aux[:, 1].tolist(),
                                    'INDICE' : data_si_dic[ cedula ]['INDICE']}
        else:
            data_val_ati[cedula] = { 'SALARIO': aux.flatten().tolist(),
                                     'ATI_CJ_M1':  [-1] * len(aux),
                                     'INDICE' : data_si_dic[ cedula ]['INDICE']} 
            
    else:
        data_val_ati[cedula] = { 'SALARIO': data_si_dic[ cedula ]['SALARIO'],
                                 'ATI_CJ_M1':  [-1] * len( data_si_dic[ cedula ]['SALARIO'] ),
                                 'INDICE' : data_si_dic[ cedula ]['INDICE']}
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  4.0  minutos y 24.964767456054688  segundos


In [10]:
inicio = time.time()
data1 = { 'CEDULA_COD': [], 'SALARIO': [], 'ATI_CJ_M1': [], 'INDICE':[]}

# Llenar las listas con los datos del diccionario
for cedula, values in data_val_ati.items():
    salario = values['SALARIO']
    atipico = values['ATI_CJ_M1']
    indice = values['INDICE']
    num_rows = len(salario)
    
    # Extender las listas en el diccionario de datos
    data1['CEDULA_COD'].extend([cedula] * num_rows)
    data1['SALARIO'].extend(salario)
    data1['ATI_CJ_M1'].extend(atipico)
    data1['INDICE'].extend(indice)

data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M1', 'INDICE']],
                          pd.DataFrame( data1 ), 
                          data_no[['CEDULA_COD', 'SALARIO', 'ATI_CJ_M1', 'INDICE']] ], axis=0)
data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
data_jerar = data_jerar.reindex(data_l.index)
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
data_jerar

KeyboardInterrupt: 

In [ ]:
print(data_jerar.shape) #(62130167, 4)
print(data_l.shape)     #(62130167, 16)

In [ ]:
data_l

In [ ]:
data_jerar

In [ ]:
inicio = time.time()
data_l['ATI_CJ_M1'] = data_jerar['ATI_CJ_M1']
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
data_l

In [ ]:
data_l = pd.concat( [ data_l, data_jerar['ATI_CJ_M1'] ], axis = 1 )

In [ ]:
del data1, data_val_ati
gc.collect()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data_dic[2778], marker='o', linestyle='-', color='b', label='Salarios')  # Graficar como línea con puntos
plt.title('Distribución de Salarios')
plt.xlabel('Índice')
plt.ylabel('Salario')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Mostrar el gráfico
plt.show()


In [ ]:
len(a)

In [ ]:
aux

In [ ]:
import numpy as np

# Ejemplo de datos auxiliares y clusters
aux = np.array([
    [161.67], [161.67], [161.67],
    [170.], [170.], [200.], [200.], [250.]
])

clusters = np.array([
    2, 2, 2,
    1, 1, 1, 1, 1
], dtype=np.int32)

# Crear un array modificado según los clusters
modified_aux = np.zeros((len(aux), 2))
modified_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
modified_aux[clusters == 1, 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1

print("Array modificado:")
print(modified_aux)

In [ ]:
plt.figure(figsize=(25, 10))
plt.title('Dendrograma')
plt.xlabel('índice de la muestra')
plt.ylabel('distancia')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
Z

In [ ]:
#Código para graficar los datos
plt.figure(figsize=(25, 10))
plt.title('Dendrograma')
plt.xlabel('índice de la muestra')
plt.ylabel('distancia')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(data_dic[2778], marker='o', linestyle='-', color='b', label='Salarios')  # Graficar como línea con puntos
plt.title('Distribución de Salarios')
plt.xlabel('Índice')
plt.ylabel('Salario')
plt.grid(True)
plt.legend()
plt.tight_layout()

# Mostrar el gráfico
plt.show()